# Deep learning with keras

First we import all required packages. Keras will import tensorflow underwater and will act as a so to speak 'interface' of the tensorflow backend.

In [5]:
import numpy as np
import keras
import random
import rasterio
import pandas as pd
from keras.utils import np_utils
import cv2
import os

Using TensorFlow backend.


This notebook contain three examples. A fully conntecte neuralnetwork, a convolutional neuralnetwork and a fully convolutional neuralnetwork.

In eacht of thesere three cases we simply define a data generator, an architecture and a loss. After that we train the architecture using the generator with the respective loss.

Lastly we validate the model on some unseen validation data.

## A fully connected neuralnetwork

We fix some helpful parameters that we might want to the tune.

In [6]:
classes = 3
epochs = 10
lr = 0.0001
split = 0.2

First we read the full iris dataset. The first four columns describe properites of the flower, the last column tells us what kind of flower we are dealing with.

In [7]:
data = pd.read_csv('iris_data.txt', header = None)
data.columns = ['prop1', 'prop2', 'prop3', 'prop4', 'type']
type_int = {'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2} 


data['type_int'] = [type_int[item] for item in data['type'].values]

indices_test = random.sample(list(np.arange(data.shape[0])), round(split*data.shape[0]))
indices_train = set(np.arange(data.shape[0])) - set(indices_test)

test_data = data.loc[list(indices_test)]
train_data = data.loc[list(indices_train)]
train_data


,prop1,prop2,prop3,prop4,type,type_int
0,5.1,3.5,1.4,0.2,Iris-setosa,0
1,4.9,3.0,1.4,0.2,Iris-setosa,0
3,4.6,3.1,1.5,0.2,Iris-setosa,0
4,5.0,3.6,1.4,0.2,Iris-setosa,0
5,5.4,3.9,1.7,0.4,Iris-setosa,0
6,4.6,3.4,1.4,0.3,Iris-setosa,0
7,5.0,3.4,1.5,0.2,Iris-setosa,0
8,4.4,2.9,1.4,0.2,Iris-setosa,0
9,4.9,3.1,1.5,0.1,Iris-setosa,0
11,4.8,3.4,1.6,0.2,Iris-setosa,0


Next we define a generator based on this table.

In [8]:
def generator(data,classes):
    while True:
        indices = np.arange(data.shape[0])
        random.shuffle(indices)
        data = data.iloc[indices]
        for i in np.arange(data.shape[0]):
           input = [data['prop1'].values[i],data['prop2'].values[i],data['prop3'].values[i],data['prop4'].values[i]]
           label =  [data['type_int'].values[i]]
           label = np_utils.to_categorical(label,classes)
           input = np.array([input])
           yield([input,label])


Lastly we define the model architecture

In [9]:
input =keras.engine.Input( shape = [4], dtype = 'float32' )

l0 = keras.layers.Dense(units = 8, activation = 'relu' )(input)
l1 = keras.layers.Dense(units = 16, activation = 'relu' )(l0)
l2 = keras.layers.Dense(units = 32, activation = 'relu' )(l1)
l3 = keras.layers.Dense(units = 16, activation = 'relu' )(l2)
l4 = keras.layers.Dense(units = 8, activation = 'relu' )(l3)
output = keras.layers.Dense(units = classes, activation = 'softmax' )(l4)

model = keras.models.Model(inputs = input, outputs = output)


In [10]:
opt = keras.optimizers.adam( lr= lr ,  clipnorm = 0.3 )
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics = ["accuracy"])

With the data generator, network architecture and loss defined we can now train the model.

In [11]:
model.fit_generator(generator = generator(train_data,classes), steps_per_epoch = train_data.shape[0], epochs = epochs)

Epoch 1/10
120/120 [==============================] - 1s 4ms/step - loss: 1.2536 - acc: 0.3417
Epoch 2/10
120/120 [==============================] - 0s 2ms/step - loss: 1.1811 - acc: 0.3417
Epoch 3/10
120/120 [==============================] - 0s 1ms/step - loss: 1.1239 - acc: 0.3417
Epoch 4/10
120/120 [==============================] - 0s 2ms/step - loss: 1.0782 - acc: 0.3417
Epoch 5/10
120/120 [==============================] - 0s 1ms/step - loss: 1.0449 - acc: 0.3417
Epoch 6/10
120/120 [==============================] - 0s 2ms/step - loss: 1.0079 - acc: 0.3417
Epoch 7/10
120/120 [==============================] - 0s 2ms/step - loss: 0.9593 - acc: 0.3417
Epoch 8/10
120/120 [==============================] - 0s 1ms/step - loss: 0.9181 - acc: 0.5583
Epoch 9/10
120/120 [==============================] - 0s 1ms/step - loss: 0.8994 - acc: 0.7000
Epoch 10/10
120/120 [==============================] - 0s 1ms/step - loss: 0.8726 - acc: 0.7000


Now let's see how well we did

In [12]:
model.evaluate_generator(generator(test_data,classes), steps = test_data.shape[0])

[0.8910885572433471, 0.5333333333333333]

## A convolutional neuralnetwork

In [13]:
classes = 62
epochs = 4
w = 60
lr = 0.0001

First we load the data, or in this case the metadata.

In [14]:
dirs = os.listdir('borden')
data = pd.DataFrame(columns = ['label','file'])

i = 0
for dir in dirs:
    files = os.listdir(os.path.join('borden',dir))
    for file in files:
        data.loc[i] = [int(dir), os.path.join('borden',dir, file)]
        i = i+1

indices_test = random.sample(list(np.arange(data.shape[0])), round(split*data.shape[0]))
indices_train = set(np.arange(data.shape[0])) - set(indices_test)

test_data = data.loc[list(indices_test)]
train_data = data.loc[list(indices_train)]
train_data


,label,file
0,7,borden/00007/37.png
1,7,borden/00007/241.png
2,7,borden/00007/1.png
3,7,borden/00007/207.png
4,7,borden/00007/165.png
5,7,borden/00007/65.png
6,7,borden/00007/141.png
7,7,borden/00007/195.png
8,7,borden/00007/99.png
10,7,borden/00007/131.png


In [19]:
def generator(data,classes):
    while True:
        indices = np.arange(data.shape[0])
        random.shuffle(indices)
        data = data.iloc[indices]
        for i in np.arange(data.shape[0]):
           input = cv2.imread(data['file'].values[i])
           input = cv2.resize(input, (w,w))
           label =  [data['label'].values[i]]
           label = np_utils.to_categorical(label,classes)
           input = np.expand_dims(input, axis = 0)
           yield([input,label])


In [21]:
input =keras.engine.Input( shape = [w,w,3], dtype = 'float32' )

l0 = keras.layers.convolutional.Conv2D( filters=16, kernel_size= (3,3),padding="same", activation = 'relu' )(input)
l1 = keras.layers.MaxPool2D(pool_size = (2,2))(l0)
l2 = keras.layers.convolutional.Conv2D( filters=32, kernel_size= (3,3),padding="same", activation = 'relu' )(l1)
l3 = keras.layers.MaxPool2D(pool_size = (2,2))(l2)
l4 = keras.layers.convolutional.Conv2D( filters=64, kernel_size= (3,3),padding="same", activation = 'relu' )(l3)
l5 = keras.layers.Flatten()(l4)
l6 = keras.layers.Dense(units = 128, activation = 'relu' )(l5)
l7 = keras.layers.Dense(units = 256, activation = 'relu' )(l6)
output = keras.layers.Dense(units = classes, activation = 'softmax' )(l7)

model = keras.models.Model(inputs = input, outputs = output)


In [22]:
opt = keras.optimizers.adam( lr= lr ,  clipnorm = 0.3 )
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics = ["accuracy"])

In [24]:
model.fit_generator(generator = generator(train_data,classes), steps_per_epoch = train_data.shape[0], epochs = epochs)

Epoch 1/4
 165/5522 [..............................] - ETA: 2:27 - loss: 15.7274 - acc: 0.0242

KeyboardInterrupt: 

In [25]:
model.evaluate_generator(generator(test_data,classes), steps = test_data.shape[0])

[15.791061580699424, 0.020289855072463767]

## A fully convolutional neuralnetwork

In [26]:
classes = 2
epochs = 4
w = 512
lr = 0.0001

In [27]:
files = os.listdir('clouds/labels')
images = [os.path.join('clouds/images', file.replace('.txt', '.jpg')) for file in files]
labels = [os.path.join('clouds/labels',file) for file in files]

data = pd.DataFrame({'image': images, 'label': labels})
indices_test = random.sample(list(np.arange(data.shape[0])), round(split*data.shape[0]))
indices_train = set(np.arange(data.shape[0])) - set(indices_test)

test_data = data.loc[list(indices_test)]
train_data = data.loc[list(indices_train)]
train_data

,image,label
0,clouds/images/wms_ALL-BANDS-L1C_EPSG4326_4.498...,clouds/labels/wms_ALL-BANDS-L1C_EPSG4326_4.498...
1,clouds/images/wms_ALL-BANDS-L1C_EPSG4326_4.497...,clouds/labels/wms_ALL-BANDS-L1C_EPSG4326_4.497...
2,clouds/images/wms_ALL-BANDS-L1C_EPSG4326_3.456...,clouds/labels/wms_ALL-BANDS-L1C_EPSG4326_3.456...
3,clouds/images/wms_ALL-BANDS-L1C_EPSG4326_5.931...,clouds/labels/wms_ALL-BANDS-L1C_EPSG4326_5.931...
5,clouds/images/wms_ALL-BANDS-L1C_EPSG4326_5.077...,clouds/labels/wms_ALL-BANDS-L1C_EPSG4326_5.077...
6,clouds/images/wms_ALL-BANDS-L1C_EPSG4326_5.041...,clouds/labels/wms_ALL-BANDS-L1C_EPSG4326_5.041...
7,clouds/images/wms_ALL-BANDS-L1C_EPSG4326_42.33...,clouds/labels/wms_ALL-BANDS-L1C_EPSG4326_42.33...
8,clouds/images/wms_ALL-BANDS-L1C_EPSG4326_49.4_...,clouds/labels/wms_ALL-BANDS-L1C_EPSG4326_49.4_...
9,clouds/images/wms_ALL-BANDS-L1C_EPSG4326_6.978...,clouds/labels/wms_ALL-BANDS-L1C_EPSG4326_6.978...
10,clouds/images/wms_ALL-BANDS-L1C_EPSG4326_36.90...,clouds/labels/wms_ALL-BANDS-L1C_EPSG4326_36.90...


In [28]:
def generator(data,classes):
    while True:
        indices = np.arange(data.shape[0])
        random.shuffle(indices)
        data = data.iloc[indices]
        for i in np.arange(data.shape[0]):
            label = np.genfromtxt(data['label'].values[i], delimiter = ',')
            label = label.astype(np.int32)
            label = np_utils.to_categorical(label, classes)
            label = np.expand_dims(label, axis = 0)
            input = cv2.imread(data['image'].values[i])
            input = np.expand_dims(input, axis = 0)
            yield([input,label])


In [29]:
input =keras.engine.Input( shape = [w,w,3], dtype = 'float32' )

l0 = keras.layers.convolutional.Conv2D( filters=64, kernel_size= (3,3),padding="same",     activation = 'relu' )(input)
l0 = keras.layers.convolutional.Conv2D( filters=64, kernel_size= (3,3),padding="same",     activation = 'relu' )(l0)
  
l1 = keras.layers.MaxPool2D(pool_size = (2,2))(l0)
l1 = keras.layers.convolutional.Conv2D( filters=128, kernel_size= (3,3),padding="same",     activation = 'relu' )(l1)
l1 = keras.layers.convolutional.Conv2D( filters=128, kernel_size= (3,3),padding="same",     activation = 'relu' )(l1)

l2 = keras.layers.MaxPool2D(pool_size = (2,2))(l1)
l2 = keras.layers.convolutional.Conv2D( filters=256, kernel_size= (3,3),padding="same",     activation = 'relu' )(l2)
l2 = keras.layers.convolutional.Conv2D( filters=256, kernel_size= (3,3),padding="same",     activation = 'relu' )(l2)

l3 = keras.layers.MaxPool2D(pool_size = (2,2))(l2)
l3 = keras.layers.convolutional.Conv2D( filters=512, kernel_size= (3,3),padding="same",     activation = 'relu' )(l3)
l3 = keras.layers.convolutional.Conv2D( filters=512, kernel_size= (3,3),padding="same",     activation = 'relu' )(l3)

l4 = keras.layers.MaxPool2D(pool_size = (2,2))(l3)
l4 = keras.layers.convolutional.Conv2D( filters=1024, kernel_size= (3,3),padding="same",     activation = 'relu' )(l4)
l4 = keras.layers.convolutional.Conv2D( filters=1024, kernel_size= (3,3),padding="same",     activation = 'relu' )(l4)


l3_up = keras.layers.convolutional.Conv2DTranspose(filters = 512 , kernel_size=(3,3) ,strides = (2, 2), padding="same")(l4)
l3_up = keras.layers.concatenate([l3,l3_up])
l3_up = keras.layers.convolutional.Conv2D( filters=512, kernel_size= (3,3),padding="same",     activation = 'relu' )(l3_up)
l3_up = keras.layers.convolutional.Conv2D( filters=512, kernel_size= (3,3),padding="same",     activation = 'relu' )(l3_up)

l2_up = keras.layers.convolutional.Conv2DTranspose(filters = 256 , kernel_size=(3,3) ,strides = (2, 2), padding="same")(l3_up)
l2_up = keras.layers.concatenate([l2,l2_up])
l2_up = keras.layers.convolutional.Conv2D( filters=256, kernel_size= (3,3),padding="same",     activation = 'relu' )(l2_up)
l2_up = keras.layers.convolutional.Conv2D( filters=256, kernel_size= (3,3),padding="same",     activation = 'relu' )(l2_up)

l1_up = keras.layers.convolutional.Conv2DTranspose(filters = 128 , kernel_size=(3,3) ,strides = (2, 2), padding="same")(l2_up)
l1_up = keras.layers.concatenate([l1,l1_up])
l1_up = keras.layers.convolutional.Conv2D( filters=128, kernel_size= (3,3),padding="same",     activation = 'relu' )(l1_up)
l1_up = keras.layers.convolutional.Conv2D( filters=128, kernel_size= (3,3),padding="same",     activation = 'relu' )(l1_up)

l0_up = keras.layers.convolutional.Conv2DTranspose(filters = 64 , kernel_size=(3,3) ,strides = (2, 2), padding="same")(l1_up)
l0_up = keras.layers.concatenate([l0,l0_up])
l0_up = keras.layers.convolutional.Conv2D( filters=64, kernel_size= (3,3),padding="same",     activation = 'relu' )(l0_up)
l0_up = keras.layers.convolutional.Conv2D( filters=64, kernel_size= (3,3),padding="same",     activation = 'relu' )(l0_up)

output = keras.layers.convolutional.Conv2D( filters=classes, kernel_size= (1,1),padding="same",     activation = 'softmax' )(l0_up)

model = keras.models.Model(inputs = input, outputs = output)


In [30]:
opt = keras.optimizers.adam( lr= lr ,  clipnorm = 0.3 )
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics = ["accuracy"])

In [31]:
model.fit_generator(generator = generator(train_data,classes), steps_per_epoch = train_data.shape[0], epochs = epochs)

Epoch 1/4


KeyboardInterrupt: 

In [ ]:
model.evaluate_generator(generator(test_data,classes), steps = test_data.shape[0])